In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
data_path = "../../data/cull%i/model_data/" % 1

In [4]:
devtest_aa_dict = np.load(data_path + 'devtest_aa_dict.npy')[()]
devtest_cmap_dict = np.load(data_path + 'devtest_cmap_dict.npy')[()]
train_aa_dict = np.load(data_path + 'train_aa_dict.npy')[()]
train_cmap_dict = np.load(data_path + 'train_cmap_dict.npy')[()]
valid_aa_dict = np.load(data_path + 'valid_aa_dict.npy')[()]
valid_cmap_dict = np.load(data_path + 'valid_cmap_dict.npy')[()]

In [15]:
# from keras.models import Sequential

# model = Sequential()

from keras import Input
from keras.models import Model
from keras import layers

input_tensor = Input(shape=(None, 20))
x = layers.Conv1D(
    40,
    3,
    activation='relu',
)(input_tensor)

x = layers.Conv1D(
    60,
    3,
    activation='relu',
)(x)

model = Model(
    input_tensor,
    x
)

# third column is the number of feature maps
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, None, 20)          0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, None, 40)          2440      
_________________________________________________________________
conv1d_10 (Conv1D)           (None, None, 60)          7260      
Total params: 9,700
Trainable params: 9,700
Non-trainable params: 0
_________________________________________________________________


In [25]:
from keras import backend as K
from keras.engine.topology import Layer
# from keras.layers import Layer

# Need to create my own outer product layer
class OuterConcat(Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        self.name = "HII"
        
    def built(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end
        
    def call(self, x):
        return K.dot(x, self.kernel)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [26]:
input_tensor = Input(shape=(None, 20))
x = layers.Conv1D(
    40,
    3,
    activation='relu',
)(input_tensor)

x = layers.Conv1D(
    60,
    3,
    activation='relu',
)(x)

x = OuterConcat(
    10
)(x)

model = Model(
    input_tensor,
    x
)

AttributeError: 'OuterConcat' object has no attribute 'input_spec'